В данном ноутбуке я "дотренировываю" модель gpt-2 "117M" (это предтренированная модель с 117 миллионами параметров, которая дообучается на нашем датасете).

! Данный ноутбук нужно запускать из Google Colab !

Импортируем нужные модули, gpt-2 работает с tensorflow первой версии.

In [1]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from google.colab import files

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Проверяем, что сейчас мы используем GPU.

In [2]:
%tensorflow_version 1.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
import numpy as np 
import pandas as pd 
import bz2
import gc
import chardet
import re
import os

LINE_COUNT = 100000 # количество отзывов, которое мы дадим модели, т.е. размер нашего датасета
root_path = "/content/drive/My Drive/NLP/inf"

Чтобы не потерять прогресс, добавим Google Drive как хранилище, чтобы gpt2 могло складывать туда результаты и модели.

In [4]:
gpt2.mount_gdrive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Скачиваем датасет, если его ещё у нас нет в Google Drive и препарируем данные - удаляем лейблы и другой мусор вроде url.

In [5]:
file_name = root_path + '/prepared_data.txt'
if not os.path.exists(root_path) or not os.path.exists(file_name):
    print('--- Making the work folder')

    if not os.path.exists("/content/drive/My Drive/NLP"):
        print("--- Downloading the dataset")

        ! wget https://storage.googleapis.com/amazonreviews/train.ft.txt.bz2
        ! wget https://storage.googleapis.com/amazonreviews/test.ft.txt.bz2
        ! bzip2 -d train.ft.txt.bz2
        ! bzip2 -d test.ft.txt.bz2

        ! mkdir "/content/drive/My Drive/NLP"
        ! mkdir "$root_path"

        ! mv "/content/train.ft.txt" "/content/drive/My Drive/NLP"
        ! mv "/content/test.ft.txt" "/content/drive/My Drive/NLP"

    with open("/content/drive/My Drive/NLP" + '/train.ft.txt') as train_file:
        train_file_lines = train_file.readlines()

    print(len(train_file_lines))
    train_file_lines = train_file_lines[:LINE_COUNT] # inf
    print(train_file_lines[10])
    print(len(train_file_lines))

    train_labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in train_file_lines]
    train_sentences = [x.split(' ', 1)[1][:-1].lower() for x in train_file_lines]

    for i in range(len(train_sentences)):
        train_sentences[i] = re.sub('\d','0',train_sentences[i])

                                                          
    for i in range(len(train_sentences)):
        if 'www.' in train_sentences[i] or 'http:' in train_sentences[i] or 'https:' in train_sentences[i] or '.com' in train_sentences[i]:
            train_sentences[i] = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", "<url>", train_sentences[i])
            

    del train_file_lines

    gc.collect()

    with open(file_name, 'w') as prepared_file:
      for feedback in train_sentences:
        prepared_file.write("%s\n" % feedback)

    gc.collect()

--- Making the work folder
3600000
__label__1 The Worst!: A complete waste of time. Typographical errors, poor grammar, and a totally pathetic plot add up to absolutely nothing. I'm embarrassed for this author and very disappointed I actually paid for this book.

100000


Загружаем модель. Я использовал самую малую "117М", потому что обучение на других занимает очень много времени и Google Colab нередко падает из-за нехватки RAM. Для более точных результатов советую взять версию побольше (см. тут https://github.com/openai/gpt-2/issues/209)

In [0]:
model_name = "117M"
if not os.path.isdir(os.path.join("models", model_name)):
	print(f"Downloading {model_name} model...")
	# gpt2.download_gpt2(model_dir=root_path + '/models', model_name=model_name)   # model is saved into current directory under /models/774M/
	gpt2.download_gpt2(model_name=model_name)   # model is saved into current directory under /models/774M/

Начинаем сессию

In [0]:
gc.collect()
sess = gpt2.start_tf_sess()

Тренируем модель и сохраняем результаты на Google Drive. Тут все параметры никак не влияют на качество модели (кроме steps и learning rate, но есть уже хорошие значения по умолчанию и авторы уверяют что не стоит их менять), лишь на процесс обучения (эти параметры были взяты из официальной документации). В процессе обучения модель выводит пробные тексты, можно посмотреть за её эволюцией :)

In [8]:
# gc.collect()
gpt2.finetune(sess,
              file_name,
              model_name=model_name,
              # model_dir=root_path + '/models',
              # checkpoint_dir=root_path + '/checkpoint',
              max_checkpoints=5,
              print_every=10,
              save_every=500,
              sample_every=200,
              steps=1000)   # steps is max number of training steps
gpt2.copy_checkpoint_to_gdrive()

/content/drive/My Drive/NLP/inf
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/117M/model.ckpt
INFO:tensorflow:Restoring parameters from models/117M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:46<00:00, 46.26s/it]


dataset has 10163761 tokens
Training...
[10 | 28.53] loss=3.71 avg=3.71
[20 | 51.05] loss=3.50 avg=3.61
[30 | 74.18] loss=3.60 avg=3.60
[40 | 97.68] loss=3.56 avg=3.59
[50 | 120.66] loss=3.45 avg=3.56
[60 | 143.62] loss=3.54 avg=3.56
[70 | 166.81] loss=3.51 avg=3.55
[80 | 190.03] loss=3.50 avg=3.54
[90 | 213.15] loss=3.60 avg=3.55
[100 | 236.26] loss=3.51 avg=3.55
[110 | 259.41] loss=3.51 avg=3.54
[120 | 282.63] loss=3.34 avg=3.52
[130 | 305.88] loss=3.46 avg=3.52
[140 | 329.08] loss=3.54 avg=3.52
[150 | 352.26] loss=3.65 avg=3.53
[160 | 375.44] loss=3.40 avg=3.52
[170 | 398.62] loss=3.56 avg=3.52
[180 | 421.78] loss=3.63 avg=3.53
[190 | 444.94] loss=3.61 avg=3.53
[200 | 468.09] loss=3.42 avg=3.53
======== SAMPLE 1 ========
.
don't care: if you wanted to get away from this book to enjoy what everyone has been through and what you have to do to stay alive, you are going to have to waste your money. i would buy this book if i were your 00rd birthday present and never finished it. it shou

Попробуем поиграться с разными параметрами генерирования текстов, например длиной.

In [9]:
gpt2.generate(sess, length=30)
gpt2.generate(sess, length=50)
gpt2.generate(sess, length=20)

"the book was the best" - i read it, and i still love it
the best: this is the best book i've ever read
cd: this is a great cd. it is great that it is not a cd, and it is not as good as the other company. i hope that this cd is okay.
don't buy this cd: this cd is not good.
the last fellas is a must read for any fans of the show. the last fellas is


О параметре top_k сами разработчики отзываются следующим образом: 

`
top_k=0: Integer value controlling diversity. 1 means only 1 word is considered for each step (token), resulting in deterministic completions, while 40 means 40 words are considered at each step. 0 is a special setting meaning no restrictions. 40 generally is a good value.
`

In [10]:
gpt2.generate(sess, length=30, top_k=40)
gpt2.generate(sess, length=30, top_k=20)
gpt2.generate(sess, length=30, top_k=60)

"one of the better cds around" the first song on my cd, it is great.i have to admit that most of the songs were
the music is very simple and it can be enjoyed and listened to on its own. the music is very simple and you can enjoy it with any type
"this is a really good book. if you like it, you'll love this book. if not, i was expecting something that was good.


Параметр temperature означает следующее:

`:temperature=1: Float value controlling randomness in boltzmann distribution. Lower temperature results in less random completions. As the temperature approaches zero, the model will become deterministic and repetitive. Higher temperature results in more random completions.`

То есть, для отзывов непохожих на отзывы из датасета нужно выбрать температуру как можно ближе к 1. Для похожих наоборот.

In [11]:
gpt2.generate(sess, length=30, top_k=40, temperature=0.1)
gpt2.generate(sess, length=30, top_k=40, temperature=1.0)
gpt2.generate(sess, length=30, top_k=40, temperature=0.4)

"the best of the best" is a great book. i read it in the middle of the night and it was the best book i have ever
it is simply not worth it to buy this type of product.
it's bad.: it is simply not worth an extra 0 cents on the dollar
"the next generation" is a very good book, but i have to say that it is not the best book i have ever read. i have


Более подробный обзор на параметры gpt-2 см. в generating_baseline.ipynb, потому что данный ноутбук предназначен для обучения модели.


Заключение: gpt-2 несмотря на то, что я обучил модель не полном датасете и взял самую маленькую версию модели из-за нехватки ресурсов, прекрасно справляется с генерированием различного рода текстов, в частности отзывов. Отзывы выглядят очень человечными и складными. Кроме того, данную модель можно ещё несколько раз дообучать на выборке, чтобы снизить loss, что ещё улучшит качество генерирования текстов.